In [1]:
import cv2
import numpy as np
from keras.models import load_model
import pickle
from mtcnn.mtcnn import MTCNN
from numpy import load
from numpy import expand_dims

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def get_embedding(model,face_pixels):
    face_pixels = face_pixels.astype('float32')
    #mean, standard deviation
    mean,std = face_pixels.mean(),face_pixels.std()
    samples = np.expand_dims(face_pixels,axis = 0)
    yhat = model.predict(samples)
    return yhat[0]

In [3]:
detector = MTCNN()
model = pickle.load(open('predict_model.sav', 'rb'))
embedding = load_model('facenet_keras.h5')
data = load('face-dataset.npz')

/home/thinh/.local/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
# Dùng mtcnn để phát hiện các khuôn mặt
# Với mỗi khuôn mặt vừa phát hiện được, resize về kích cỡ 160x160
image = cv2.imread("1.jpg",1)

result = detector.detect_faces(image)
imgs = list()
for person in result:
    bounding_box = person['box']
    keypoints = person['keypoints']
    x = bounding_box[1]
    y = bounding_box[0]
    h = bounding_box[2]
    w = bounding_box[3]
    img = image[x:x+w,y:y+h]
    img = cv2.resize(img,(160,160))
    imgs.append(img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# Chuyển các khuôn mặt về dạng embedding vector
embedding_imgs = list()
for img in imgs:
    embedding_imgs.append(get_embedding(embedding,img))
    


In [6]:
# Dự đoán
samples = expand_dims(embedding_imgs[0],axis = 0)
# Nếu threshold < 60: Stranger

In [7]:
yhat_class = model.predict(samples)
print("yhat_class: ",yhat_class)
yhat_prob = model.predict_proba(samples)
print(yhat_prob)

yhat_class:  [0]
[[9.99999817e-01 1.00000022e-07 5.00000108e-08 3.33338835e-08]]


In [8]:
cv2.imshow("hhh",imgs[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
for i in range(len(imgs)):
    cv2.imwrite(str(i)+".jpg",imgs[i])